<a href="https://colab.research.google.com/github/Thandeka20/Sentinel-2/blob/main/S2_uMzim_ROI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount to Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import required functions**

In [ ]:
!pip install rasterio
!pip install pycrs
!pip install earthengine-api
!pip install geemap

import ee
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import geemap
import re
import os
from IPython.display import Image, display
from google.colab import drive
from google.colab import files
import shutil
import pycrs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 30.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycrs: filename=PyCRS-1.0.2-py3-none-any.whl size=32686 sha256=f8402ac7beeec7c28605c831c7b5aef2c69b180b661b7378896f52b88afd9424
  Stored in directory: /root/.cache/pip/wheels/47/1d/70/7a5bdf33347e7c75e95b06b1fa38f076a59a9506653cc24aff
Successfully built pycrs
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


**Authenticate Google Earth Engine**

In [ ]:
# Step 1: Trigger the authentication flow.
ee.Authenticate()

# Step 2: Set the environment variable for the Project ID.
os.environ['EE_PROJECT_ID'] = 'thandeka-skosana'

# Step 3: Initialize the library.
ee.Initialize(project=os.getenv('EE_PROJECT_ID'))


**Map interface for visualization**

In [ ]:
Map=geemap.Map()

**Load ROI and filter Sentinel-2 image collection**

In [ ]:
# Load the shapefile representing the AOI
shapefile_path = '/content/drive/MyDrive/Colab Notebooks/uMzim_ROI/roi/ROI.shp'
geometry = geemap.shp_to_ee(shapefile_path)

# Filter the start and end dates for your image collection.
start_date = '2023-05-01'
end_date = '2023-06-19'

# Filter for cloud cover.
cloud_filter = ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT', 0.1)

# Load the Sentinel 2 imagery collection.
s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                  .filterDate(start_date, end_date)
                  .filterBounds(geometry)
                  .filter(cloud_filter)
                  .sort('system:time_start', True))

mosaic= s2_collection.mosaic()

# Clip the imagery to the AOI
umzimvubu_clipped = mosaic.clip(geometry)

**Load training data from google drive and convert to a feature collection**

In [ ]:
# Load training data
file_path = '/content/drive/MyDrive/Colab Notebooks/uMzim_ROI/training/train_shapefile.shp'
training_data = geemap.shp_to_ee(file_path)
num_points = training_data.size()
print("Number of points in training data:", num_points.getInfo())
Map.addLayer(training_data)

Number of points in training data: 168


**Visualize the S2 collection on the map interface**

In [ ]:
# Visualize the Sentinel-2 image collection
bands = ee.List(["B4", "B3", "B2"])
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
}
Map.addLayer(umzimvubu_clipped, vis_params, 'Sentinel-2 Image')
#Map.centerObject(umzimvubu_clipped, zoom=8)
Map.addLayer(geometry, {'color': 'red', 'lineWidth': 2}, 'uMzim_Boundary')
Map.addLayer(training_data)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

**Rescale bands**

In [ ]:
# Rescale image bands / 10,000
sentinel2_image = umzimvubu_clipped.divide(10000)

**Compute vegetation indices and add to the image collection**

In [ ]:
def add_indices(image):
    GI = image.expression('(GREEN)/(RED)', {
        'GREEN': image.select('B3'),
        'RED': image.select('B4'),
    }).rename('GI')

    IRG = image.expression('(RED-GREEN)', {
        'RED': image.select('B4'),
        'GREEN': image.select('B3'),
    }).rename('IRG')

    NGRDI = image.expression('(GREEN-RED)/(GREEN+RED)', {
        'RED': image.select('B4'),
        'GREEN': image.select('B3'),
    }).rename('NGRDI')

    VARI = image.expression('(GREEN-RED)/(GREEN+RED+BLUE)', {
        'RED': image.select('B4'),
        'GREEN': image.select('B3'),
        'BLUE': image.select('B2'),
    }).rename('VARI')

    VDVI = image.expression('(2*GREEN-RED-BLUE)/(2*GREEN+RED+BLUE)', {
        'RED': image.select('B4'),
        'GREEN': image.select('B3'),
        'BLUE': image.select('B2'),
    }).rename('VDVI')

    return image.addBands([GI, IRG, NGRDI, VARI, VDVI])

# Apply the add_indices function to the single image
umzimvubu_indices = add_indices(sentinel2_image)

# Get the band names of the modified image
band_names = umzimvubu_indices.bandNames()

# Convert the band names to a Python list
band_names_list = band_names.getInfo()

# Print the band names
print("Band names:", band_names_list)


Band names: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'MSK_CLASSI_OPAQUE', 'MSK_CLASSI_CIRRUS', 'MSK_CLASSI_SNOW_ICE', 'GI', 'IRG', 'NGRDI', 'VARI', 'VDVI']


**Select bands to use in classification**

In [ ]:
#select bands
# Define the bands to select
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'GI', 'IRG', 'NGRDI', 'VARI', 'VDVI']

# Sample regions from the image at training feature locations
training_dataset = umzimvubu_indices.select(bands).sampleRegions(
    collection=training_data,
    properties=['ID', 'LULC'],
    scale=10
)

# Export the training dataset to Google Drive for use in wavelength plot
task = ee.batch.Export.table.toDrive(
    collection=training_dataset,
    description='Training_BandsIndices',
    fileFormat='CSV'
)

# Start the export task
task.start()

# Inform the user that the task has started
print('Export task started. Monitor the task in the Earth Engine Task Manager.')

Export task started. Monitor the task in the Earth Engine Task Manager.


**Run the Random Forest classifier**

In [ ]:
#Classification
classifier = ee.Classifier.smileRandomForest(500).train(
    features=training_dataset,
    classProperty='ID',
    inputProperties=bands
)
classified = umzimvubu_indices.select(bands).classify(classifier)

**Visualize classification**

In [ ]:
# Define the palette
uMzimvubuPalette = [
    'FF0000', 'FFAA00', 'FF00C5', 'ffffff', 'a8a800',
    '6aa84f', '783f04', '000000'
]

# Visualization parameters
viz = {
    'min': 1,
    'max': 8,
    'palette': uMzimvubuPalette
}

# Create a Map object using geemap
m = geemap.Map(center=[20, 0], zoom=3)

# Add the classified image to the map

m.addLayer(classified, viz, 'Classification')
#m.centerObject(classified, zoom=9)

# Display the legend
names = ['Alien_Black Wattle','Alien_Gum', 'Alien_Silver Wattle', 'Bare Ground', 'Grassland', 'Indigenous Bush', 'Maize','Urban' ]
m.add_legend(title='Classification', colors=uMzimvubuPalette, labels=names)

# Display the map
m

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…

**Export classification**

In [ ]:
# Convert the FeatureCollection to a Geometry
ROI = geometry.geometry()

# Now you can use this geometry for exporting the image
task = ee.batch.Export.image.toDrive(
    image=classified,
    description='S2_Classified_final',
    crs='EPSG:32735',
    scale=10,
    maxPixels=1e9,
    region=ROI
)

# Start the export task
task.start()

print('Export task started. Monitor the task in the Earth Engine Task Manager.')

Export task started. Monitor the task in the Earth Engine Task Manager.
